## Script de extração de dados do JusBrasil

In [51]:
import os
import pandas as pd
import numpy as np
import camelot
import matplotlib.pyplot as plt
import matplotlib
import requests
import bs4
from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text
import tempfile
import fitz

In [28]:
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [29]:
anexos_copia_min = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Tribunal"]]
sentencas = anexos_copia_min.loc[anexos["Tipo de anexo"] == "SENTENCA", ["processoID", "processoAnexoID", "Download copia", "Tribunal"]]

In [67]:
sentencas

,processoID,processoAnexoID,Download copia,Tribunal
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,TJAM
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,TJAM
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,TJAM
...,...,...,...,...
13643,494408691,2315018026,http://jud-anexos.digesto.com.br/8ab01c9d05320...,TRF1
13876,494410810,269258456,http://jud-anexos.digesto.com.br/e55b90bdcd194...,TRF1
13877,494410810,2273754745,http://jud-anexos.digesto.com.br/e3f977a11a4d1...,TRF1
13903,549928831,1874259862,http://jud-anexos.digesto.com.br/4c97bd2fdb061...,TRF1


In [74]:
sucessos = 0
processos = []

for i in range(sentencas.shape[0]):
    link = sentencas.iloc[i, 2]
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    texto = ""
    num_processo = " "

    if 'pdf' in content_type:
        # print(f"[PDF] Extraindo de: {link}")
        # Criando arquivo temporário para ler conteúdo do PDF 
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        # Abrindo arquivo temporário e extraindo texto 
        with fitz.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
        
        index_num_proc = texto.find("Processo:")
        if index_num_proc != -1:
            # Indica que encontrou na string o número do processo
            num_processo = texto[index_num_proc+10:index_num_proc+35].strip()
            sucessos += 1

    elif 'html' in content_type:
        # print(f"[HTML] Extraindo de: {link}")
        soup = BeautifulSoup(response.content, 'html.parser')
        texto = soup.get_text(separator='\n')

        index_num_proc1 = texto.find("Processo n.:")
        index_num_proc2 = texto.find("Processo nº:")
        if index_num_proc1 != -1:
            num_processo = texto[index_num_proc1+13:index_num_proc1+38].strip()
            sucessos += 1
        if index_num_proc2 != -1:
            num_processo = texto[index_num_proc2+13:index_num_proc2+38].strip()
            sucessos += 1
    
    processos.append(num_processo)

print(f"Números de processos capturados: {sucessos}")
sentencas["num_processo"] = processos
sentencas

Números de processos capturados: 105


,processoID,processoAnexoID,Download copia,Tribunal,num_processo
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM,0600357-89.2021.8.04.5600
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM,0600384-72.2021.8.04.5600
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,TJAM,0681058-19.2022.8.04.0001
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,TJAM,0471756-76.2024.8.04.0001
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,TJAM,0097690-14.2024.8.04.1000
...,...,...,...,...,...
13643,494408691,2315018026,http://jud-anexos.digesto.com.br/8ab01c9d05320...,TRF1,
13876,494410810,269258456,http://jud-anexos.digesto.com.br/e55b90bdcd194...,TRF1,
13877,494410810,2273754745,http://jud-anexos.digesto.com.br/e3f977a11a4d1...,TRF1,
13903,549928831,1874259862,http://jud-anexos.digesto.com.br/4c97bd2fdb061...,TRF1,
